Address, City, Zip code
Number of bedrooms and bathrooms
Size of the house and Square footage
Year built
Sale price 

In [10]:
#Importing packages
from bs4 import BeautifulSoup 
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
import regex as re
import time
import numpy as np
import pandas as pd

In [11]:
executable_path = {'executable_path': 'C:\\Users\\debor\\GWARL201811DATA3\\01-Class-Activities\\12-Web-Scraping-and-Document-Databases\\chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [12]:
url='https://www.zillow.com/alexandria-va/houses/'
browser.visit(url)


In [32]:
html = browser.html
soup = BeautifulSoup(html, "html.parser")
listings = soup.find_all("a", class_="zsg-photo-card-overlay-link")
listings[:5]

[<a class="zsg-photo-card-overlay-link routable hdp-link routable mask hdp-link" href="/homedetails/229-N-Henry-St-Alexandria-VA-22314/192120889_zpid/"></a>,
 <a class="zsg-photo-card-overlay-link routable hdp-link show-lightbox mask" data-after-auth-action-type="Event" data-after-auth-event="authSuccessNotify" data-za-label="Camo Map List" href="/homedetail/AuthRequired.htm?r=pfc&amp;zpid=249463575" id="register_opener" rel="nofollow"></a>,
 <a class="zsg-photo-card-overlay-link routable hdp-link routable mask hdp-link" href="/homedetails/4431-Venable-Ave-Alexandria-VA-22304/192133410_zpid/"></a>,
 <a class="zsg-photo-card-overlay-link routable hdp-link routable mask hdp-link" href="/homedetails/3301-King-St-Alexandria-VA-22302/192132448_zpid/"></a>,
 <a class="zsg-photo-card-overlay-link routable hdp-link routable mask hdp-link" href="/homedetails/2508-Leslie-Ave-Alexandria-VA-22301/192125357_zpid/"></a>]

In [33]:

house_links = ['https://www.zillow.com/alexandria-va/houses/'+ row['href'] for row in listings]
print(house_links)

['https://www.zillow.com/alexandria-va/houses//homedetails/229-N-Henry-St-Alexandria-VA-22314/192120889_zpid/', 'https://www.zillow.com/alexandria-va/houses//homedetail/AuthRequired.htm?r=pfc&zpid=249463575', 'https://www.zillow.com/alexandria-va/houses//homedetails/4431-Venable-Ave-Alexandria-VA-22304/192133410_zpid/', 'https://www.zillow.com/alexandria-va/houses//homedetails/3301-King-St-Alexandria-VA-22302/192132448_zpid/', 'https://www.zillow.com/alexandria-va/houses//homedetails/2508-Leslie-Ave-Alexandria-VA-22301/192125357_zpid/', 'https://www.zillow.com/alexandria-va/houses//homedetails/3655-Trinity-Dr-Alexandria-VA-22304/192133694_zpid/', 'https://www.zillow.com/alexandria-va/houses//homedetails/109-Ashby-St-Alexandria-VA-22305/192125680_zpid/', 'https://www.zillow.com/alexandria-va/houses//homedetails/110-S-Pitt-St-Alexandria-VA-22314/2085545498_zpid/', 'https://www.zillow.com/alexandria-va/houses//homedetails/620-Mckenzie-Ave-Alexandria-VA-22301/2085762654_zpid/', 'https://ww

In [36]:

def visit_house_links(url, browser, pages=3):
    house_links=[]
    browser.visit(url)
    
    for i in range(pages):
        soup = BeautifulSoup(browser.page_source, 'html.parser')
        listings = soup.find_all("a", class_="zsg-photo-card-overlay-link")
        page_data = ['https://www.zillow.com'+row['href'] for row in listings]
        house_links.append(page_data)
        house_links = ['https://www.zillow.com/alexandria-va/houses'+ row['href'] for row in listings]
        print(house_links)

        time.sleep(np.random.lognormal(0,1))
        next_button = soup.find_all("a", class_="on")
        next_button_link = ['https://www.zillow.com'+row['href'] for row in next_button]
        if i<2:
            browser.visit(next_button_link[0])
        return house_links


In [41]:
#Scrape price data
html = browser.html
soup = BeautifulSoup(html, "html.parser")
price=[]



[<span class="zsg-photo-card-price">$529,900</span>, <span class="zsg-photo-card-price">$450,000</span>, <span class="zsg-photo-card-price">$599,900</span>, <span class="zsg-photo-card-price">$1,199,000</span>, <span class="zsg-photo-card-price">$799,900</span>, <span class="zsg-photo-card-price">$999,000</span>, <span class="zsg-photo-card-price">$3,750,000</span>, <span class="zsg-photo-card-price">$835,000</span>, <span class="zsg-photo-card-price">$682,000</span>, <span class="zsg-photo-card-price">$1,399,000</span>, <span class="zsg-photo-card-price">$989,000</span>, <span class="zsg-photo-card-price">$569,500</span>, <span class="zsg-photo-card-price">$939,000</span>, <span class="zsg-photo-card-price">$1,775,000</span>, <span class="zsg-photo-card-price">$1,099,000</span>, <span class="zsg-photo-card-price">$549,900</span>, <span class="zsg-photo-card-price">$775,000</span>, <span class="zsg-photo-card-price">$6,900,000</span>, <span class="zsg-photo-card-price">$565,000</span>,

In [69]:
#Append it to a list
html = browser.html
soup = BeautifulSoup(html, "html.parser")
elements = soup.find_all(class_="price")
for e in elements:
    try:
        price = element.find_all("span")[1].text
        price = price.replace(",", "").replace("+", "").replace("$", "").lower()
        print(int(price))
    except ElementDoesNotExist as exception:
        print("Nan") 

        
#price = soup.find('span', {'class': 'zsg-photo-card-price'}).text
#info = soup.find('span', {'class': 'zsg-photo-card-info'}).text
#address = soup.find('span', {'itemprop': 'address'}).text

In [70]:
price_df=pd.DataFrame(price)
price_df
price_df.columns = ["Price"]
price_df

,Price
0,"$529,900"
1,"$450,000"
2,"$599,900"
3,"$1,199,000"
4,"$799,900"
5,"$999,000"
6,"$3,750,000"
7,"$835,000"
8,"$682,000"
9,"$1,399,000"


In [51]:
#Scrape info data including no.of bedrooms, bathrooms and sqft.
soup = BeautifulSoup(html, "html.parser")
info=[]
details=soup.find_all('span', {'class': 'zsg-photo-card-info'})
print(details)

[<span class="zsg-photo-card-info">3 bds <span class="interpunct">·</span> 2 ba <span class="interpunct">·</span> 1,208 sqft</span>, <span class="zsg-photo-card-info">Studio <span class="interpunct">·</span> 1 ba <span class="interpunct">·</span> 1,024 sqft</span>, <span class="zsg-photo-card-info">3 bds <span class="interpunct">·</span> 2 ba <span class="interpunct">·</span> 1,367 sqft</span>, <span class="zsg-photo-card-info">5 bds <span class="interpunct">·</span> 3 ba <span class="interpunct">·</span> 2,400 sqft</span>, <span class="zsg-photo-card-info">4 bds <span class="interpunct">·</span> 3 ba <span class="interpunct">·</span> 1,665 sqft</span>, <span class="zsg-photo-card-info">3 bds <span class="interpunct">·</span> 3 ba <span class="interpunct">·</span> 2,269 sqft</span>, <span class="zsg-photo-card-info">5 bds <span class="interpunct">·</span> 4 ba <span class="interpunct">·</span> 3,950 sqft</span>, <span class="zsg-photo-card-info">6 bds <span class="interpunct">·</span> 

In [52]:
#Append it to a list
for d in details:
    try:
        details=d.text
        info.append(details)
    except ElementDoesNotExist as exception:
        print("Element does not Exist")

In [53]:
info


['3 bds · 2 ba · 1,208 sqft',
 'Studio · 1 ba · 1,024 sqft',
 '3 bds · 2 ba · 1,367 sqft',
 '5 bds · 3 ba · 2,400 sqft',
 '4 bds · 3 ba · 1,665 sqft',
 '3 bds · 3 ba · 2,269 sqft',
 '5 bds · 4 ba · 3,950 sqft',
 '6 bds · 7 ba · 10,112 sqft',
 '3 bds · 3 ba · 2,721 sqft',
 '3 bds · 2 ba · 2,295 sqft',
 '7 bds · 7 ba · 4,050 sqft',
 '4 bds · 3 ba · 2,400 sqft',
 '3 bds · 2 ba · 1,175 sqft',
 '4 bds · 4 ba · 2,770 sqft',
 '6 bds · 5 ba · 4,965 sqft',
 '3 bds · 3 ba · 1,904 sqft',
 '5 bds · 2 ba · 2,340 sqft',
 '3 bds · 2 ba · 1,777 sqft',
 '6 bds · 5 ba · 8,145 sqft',
 '3 bds · 2 ba · 1,200 sqft',
 '3 bds · 2 ba · 1,178 sqft',
 '5 bds · 4 ba · 4,100 sqft',
 '3 bds · 1.5 ba · 1,324 sqft',
 '6 bds · 6 ba · 6,867 sqft',
 '6 bds · 4.5 ba · 5,259 sqft']

In [54]:
#Scrape address data
html = browser.html
soup = BeautifulSoup(html, "html.parser")
address=[]
location = soup.find_all('span', {'itemprop': 'address'})
print(location)

[<span class="hide" itemprop="address" itemscope="" itemtype="http://schema.org/PostalAddress"><span itemprop="streetAddress">229 N Henry St</span><span itemprop="addressLocality"> Alexandria </span><span itemprop="addressRegion">VA </span><span class="hide" itemprop="postalCode">22314</span></span>, <span class="hide" itemprop="address" itemscope="" itemtype="http://schema.org/PostalAddress"><span itemprop="streetAddress">Edison St</span><span itemprop="addressLocality"> Alexandria </span><span itemprop="addressRegion">VA </span><span class="hide" itemprop="postalCode">22305</span></span>, <span class="hide" itemprop="address" itemscope="" itemtype="http://schema.org/PostalAddress"><span itemprop="streetAddress">4431 Venable Ave</span><span itemprop="addressLocality"> Alexandria </span><span itemprop="addressRegion">VA </span><span class="hide" itemprop="postalCode">22304</span></span>, <span class="hide" itemprop="address" itemscope="" itemtype="http://schema.org/PostalAddress"><span

In [55]:
#Append it to a list
for l in location:
    try:
        location=l.text
        address.append(location)
    except ElementDoesNotExist as exception:
        print("Element Does Not Exist")  
          

In [56]:
html = browser.html
soup = BeautifulSoup(html, "html.parser")
brokers=[] 
brokers_name= soup.find_all('span', {'class': 'zsg-photo-card-broker-name'})
print(brokers_name)
              

[<span class="zsg-photo-card-broker-name">Coldwell Banker Elite</span>, <span class="zsg-photo-card-broker-name">TTR Sotheby's International Realty</span>, <span class="zsg-photo-card-broker-name">CENTURY 21 New Millennium</span>, <span class="zsg-photo-card-broker-name">Coldwell Banker Residential Brokerage - Fairfax</span>, <span class="zsg-photo-card-broker-name">TTR Sotheby's International Realty</span>]


In [57]:
for br in brokers_name:
    try:
        brokers_name= br.text
        brokers.append(brokers_name)
    except ElementDoesNotExist as exception:
        print("Element does not Exist")     

In [71]:
index=[]
data = { 'House Url': house_links, 'Facts and Features': info, 'Address': address}
if index is None:
    index=extract_index(arrays)
house_data= pd.DataFrame(data)

print(len(price))
print(len(house_links))
print(len(info))
print(len(address))





24
25
25
25


In [72]:
house_data

,House Url,Facts and Features,Address
0,https://www.zillow.com/alexandria-va/houses//h...,"3 bds · 2 ba · 1,208 sqft",229 N Henry St Alexandria VA 22314
1,https://www.zillow.com/alexandria-va/houses//h...,"Studio · 1 ba · 1,024 sqft",Edison St Alexandria VA 22305
2,https://www.zillow.com/alexandria-va/houses//h...,"3 bds · 2 ba · 1,367 sqft",4431 Venable Ave Alexandria VA 22304
3,https://www.zillow.com/alexandria-va/houses//h...,"5 bds · 3 ba · 2,400 sqft",3301 King St Alexandria VA 22302
4,https://www.zillow.com/alexandria-va/houses//h...,"4 bds · 3 ba · 1,665 sqft",2508 Leslie Ave Alexandria VA 22301
5,https://www.zillow.com/alexandria-va/houses//h...,"3 bds · 3 ba · 2,269 sqft",3655 Trinity Dr Alexandria VA 22304
6,https://www.zillow.com/alexandria-va/houses//h...,"5 bds · 4 ba · 3,950 sqft",109 Ashby St Alexandria VA 22305
7,https://www.zillow.com/alexandria-va/houses//h...,"6 bds · 7 ba · 10,112 sqft",110 S Pitt St Alexandria VA 22314
8,https://www.zillow.com/alexandria-va/houses//h...,"3 bds · 3 ba · 2,721 sqft",620 Mckenzie Ave Alexandria VA 22301
9,https://www.zillow.com/alexandria-va/houses//h...,"3 bds · 2 ba · 2,295 sqft",1001 Woods Pl Alexandria VA 22302


In [74]:
house_data.to_csv('Alexandria Home Listings.csv')